In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

41751


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [91]:
add_histone = (
    "INSERT INTO histone "
    "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
    "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
)
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
add_sequence_has_publication = (
    "INSERT INTO sequence_has_publication "
    "(sequence_accession, publication_id) "
    "VALUES (%s, %s)"
)
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

In [7]:
query = "SELECT * FROM alternative_name " "WHERE name='cid'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,65,cid,None,None,None,cenH3
1,93,cid,None,None,None,cenH3_(Animals)


# To Do

## <span style="color:green">Add to gene description of cenH3_(Plants):</span>

```
It was demonstrated that the duplication of HTR12 in A. lyrata may have facilitated adaptation to multiple centromeric satellite sequences, though the underlying mechanisms of this process require further investigation [kawabe_duplication_2006]. Pisum and closely related Lathyrus species also retain two CenH3 paralogs (CenH3-1 and CenH3-2) originating from a Fabeae ancestor [neumann_centromeres_2015, neumann_stretching_2012]. Despite divergence, they colocalize in centromeres, likely enabling meta-polycentric organization with multiple functional domains. CenH3-2 is highly conserved but shows adaptive evolution in its histone fold, while CenH3-1 is more variable (especially N-terminal) without signs of positive selection [neumann_centromeres_2015]. The diploid barley species H. vulgare and H. bulbosum each encode two functional CENH3 variants (α and β) that arose before their divergence (~7 mya), are expressed and incorporated into centromeres, but display distinct interspecific incorporation patterns in hybrids [sanei_loss_2011, karimi-ashtiyani_point_2015]. Notably, in early embryos, some centromeres (9.6%) contain only βCENH3, while later stages always incorporate both variants, revealing developmental-stage-specific loading [ishii_differential_2015]. In rye, both CENH3 genes (α and β) are actively transcribed, retained in the genome, and likely functionally important for centromeric nucleosome assembly, while alternative splicing of αCENH3 generates protein diversity without affecting key DNA-binding domains (CATD) [evtushenko_conserved_2017].
```

## <span style="color:green">Add to structure description of cenH3_(Plants):</span>
```
The L92F mutation in barley βCENH3 (and its orthologous variants in other plants) disrupts centromeric histone loading, enabling the development of haploid inducer lines for accelerated breeding, as it triggers selective elimination of mutant-derived chromosomes in hybrids with wild-type parents [karimi-ashtiyani_point_2015].
```

## <span style="color:green">Add summary description of cenH3_(Mammalia):</span>

```
cenH3_(Mammalia) is a centromere-specific histone variant in mammals, often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation. For a general description see cenH3_(Animals) class.
```

## <span style="color:green">Add to gene description of cenH3_(Mammalia):</span>

```
Eleven centromeric histone CENP-A genes were identified in cattle (Bos taurus), among which only CENP-A-L-1 appears to be functionally active, while the remaining members likely represent nonfunctional pseudogenes [li_identification_2008].
```

## <span style="color:green">Add summary description of cenH3_(Animals):</span>

```
cenH3_(Animals) is a centromere-specific histone variant in animals (Metazoa), often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation.
```

## <span style="color:green">Add to function description of cenH3_(Animals):</span>

```
CENP-A nucleosomes are essential for chromosome segregation. A functional kinetochore interacts with active centromeric chromatin reached with CENP-A to form the mitotic spindle. Notably, in certain holocentric organisms, CENP-A appears dispensable for meiotic chromosome segregation while remaining essential for mitosis. As demonstrated in Caenorhabditis elegans, CENP-A loading is specifically eliminated following meiosis I, and RNAi-mediated depletion of CENP-A during meiosis fails to disrupt proper chromosome segregation [monen_differential_2005]. This stands in striking contrast to mitotic divisions, where CENP-A is absolutely required for kinetochore assembly and faithful chromosome segregation. Moreover, C. elegans and C. remanei possess a second cenH3 histone gene that likely performs specialized functions distinct from canonical CENP-A roles [monen_separase_2015]. For more detailed information see the "gene" section.
```

## <span style="color:green">Add to gene description of cenH3_(Animals):</span>

```
The nematodes Caenorhabditis elegans and C. remanei possess two genes encoding the centromeric histone H3 variant (cenH3): HCP-3 (also known as CeCENP-A) and CPAR-1 [monen_separase_2015, monen_differential_2005]. While HCP-3 functions as the predominant centromeric histone - being highly expressed and essential for precise chromosome segregation during mitotic divisions - CPAR-1 displays distinct characteristics: it exhibits lower expression levels but shows specific enrichment on meiotic chromosomes. Importantly, CPAR-1 undergoes separase-mediated proteolytic cleavage at the meiosis I metaphase-to-anaphase transition, implying a specialized, though not yet fully characterized, role in meiotic processes [monen_separase_2015]. 
```


# <span style="color:black">Add to gene description of cenH3_(Plants):</span>

```
It was demonstrated that the duplication of HTR12 in A. lyrata may have facilitated adaptation to multiple centromeric satellite sequences, though the underlying mechanisms of this process require further investigation [kawabe_duplication_2006]. Pisum and closely related Lathyrus species also retain two CenH3 paralogs (CenH3-1 and CenH3-2) originating from a Fabeae ancestor [neumann_centromeres_2015, neumann_stretching_2012]. Despite divergence, they colocalize in centromeres, likely enabling meta-polycentric organization with multiple functional domains. CenH3-2 is highly conserved but shows adaptive evolution in its histone fold, while CenH3-1 is more variable (especially N-terminal) without signs of positive selection [neumann_centromeres_2015]. The diploid barley species H. vulgare and H. bulbosum each encode two functional CENH3 variants (α and β) that arose before their divergence (~7 mya), are expressed and incorporated into centromeres, but display distinct interspecific incorporation patterns in hybrids [sanei_loss_2011, karimi-ashtiyani_point_2015]. Notably, in early embryos, some centromeres (9.6%) contain only βCENH3, while later stages always incorporate both variants, revealing developmental-stage-specific loading [ishii_differential_2015]. In rye, both CENH3 genes (α and β) are actively transcribed, retained in the genome, and likely functionally important for centromeric nucleosome assembly, while alternative splicing of αCENH3 generates protein diversity without affecting key DNA-binding domains (CATD) [evtushenko_conserved_2017].
```

In [12]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,233,cenH3_(Plants) is a centromere-specific histon...,None,While most diploid eukaryotes and flowering pl...,...,None,None,None,None,None,None,None,None,None,None


In [13]:
df["genes"].values[0]

'While most diploid eukaryotes and flowering plants maintain only one CENH3 gene copy (even after whole-genome duplications), some diploid species like Arabidopsis lyrata, barley, rye, pea, and related legumes retain two functional CENH3 homologs [ishii_unequal_2020].'

In [16]:
gene_desc = (
    df["genes"].values[0]
    + " It was demonstrated that the duplication of HTR12 in A. lyrata may have facilitated adaptation to multiple centromeric satellite sequences, though the underlying mechanisms of this process require further investigation [kawabe_duplication_2006]. Pisum and closely related Lathyrus species also retain two CenH3 paralogs (CenH3-1 and CenH3-2) originating from a Fabeae ancestor [neumann_centromeres_2015, neumann_stretching_2012]. Despite divergence, they colocalize in centromeres, likely enabling meta-polycentric organization with multiple functional domains. CenH3-2 is highly conserved but shows adaptive evolution in its histone fold, while CenH3-1 is more variable (especially N-terminal) without signs of positive selection [neumann_centromeres_2015]. The diploid barley species H. vulgare and H. bulbosum each encode two functional CENH3 variants (α and β) that arose before their divergence (~7 mya), are expressed and incorporated into centromeres, but display distinct interspecific incorporation patterns in hybrids [sanei_loss_2011, karimi-ashtiyani_point_2015]. Notably, in early embryos, some centromeres (9.6%) contain only βCENH3, while later stages always incorporate both variants, revealing developmental-stage-specific loading [ishii_differential_2015]. In rye, both CENH3 genes (α and β) are actively transcribed, retained in the genome, and likely functionally important for centromeric nucleosome assembly, while alternative splicing of αCENH3 generates protein diversity without affecting key DNA-binding domains (CATD) [evtushenko_conserved_2017]."
)
query = f"UPDATE histone_description SET genes='{gene_desc}' WHERE id=233"
print(query)
cursor.execute(query)

UPDATE histone_description SET genes='While most diploid eukaryotes and flowering plants maintain only one CENH3 gene copy (even after whole-genome duplications), some diploid species like Arabidopsis lyrata, barley, rye, pea, and related legumes retain two functional CENH3 homologs [ishii_unequal_2020]. It was demonstrated that the duplication of HTR12 in A. lyrata may have facilitated adaptation to multiple centromeric satellite sequences, though the underlying mechanisms of this process require further investigation [kawabe_duplication_2006]. Pisum and closely related Lathyrus species also retain two CenH3 paralogs (CenH3-1 and CenH3-2) originating from a Fabeae ancestor [neumann_centromeres_2015, neumann_stretching_2012]. Despite divergence, they colocalize in centromeres, likely enabling meta-polycentric organization with multiple functional domains. CenH3-2 is highly conserved but shows adaptive evolution in its histone fold, while CenH3-1 is more variable (especially N-terminal)

In [18]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])[
    "genes"
].values[0]

'While most diploid eukaryotes and flowering plants maintain only one CENH3 gene copy (even after whole-genome duplications), some diploid species like Arabidopsis lyrata, barley, rye, pea, and related legumes retain two functional CENH3 homologs [ishii_unequal_2020]. It was demonstrated that the duplication of HTR12 in A. lyrata may have facilitated adaptation to multiple centromeric satellite sequences, though the underlying mechanisms of this process require further investigation [kawabe_duplication_2006]. Pisum and closely related Lathyrus species also retain two CenH3 paralogs (CenH3-1 and CenH3-2) originating from a Fabeae ancestor [neumann_centromeres_2015, neumann_stretching_2012]. Despite divergence, they colocalize in centromeres, likely enabling meta-polycentric organization with multiple functional domains. CenH3-2 is highly conserved but shows adaptive evolution in its histone fold, while CenH3-1 is more variable (especially N-terminal) without signs of positive selection 

In [19]:
# Make sure data is committed to the database
conn.commit()

# <span style="color:black">Add to structure description of cenH3_(Plants):</span>
```
The L92F mutation in barley βCENH3 (and its orthologous variants in other plants) disrupts centromeric histone loading, enabling the development of haploid inducer lines for accelerated breeding, as it triggers selective elimination of mutant-derived chromosomes in hybrids with wild-type parents [karimi-ashtiyani_point_2015].
```

In [20]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,233,cenH3_(Plants) is a centromere-specific histon...,None,While most diploid eukaryotes and flowering pl...,...,None,None,None,None,None,None,None,None,None,None


In [21]:
df["structure"].values[0]

In [26]:
structure_desc = "The L92F mutation in barley βCENH3 (and its orthologous variants in other plants) disrupts centromeric histone loading, enabling the development of haploid inducer lines for accelerated breeding, as it triggers selective elimination of mutant-derived chromosomes in hybrids with wild-type parents [karimi-ashtiyani_point_2015]."
query = f"UPDATE histone_description SET structure='{structure_desc}' WHERE id=233"
print(query)
cursor.execute(query)

UPDATE histone_description SET structure='The L92F mutation in barley βCENH3 (and its orthologous variants in other plants) disrupts centromeric histone loading, enabling the development of haploid inducer lines for accelerated breeding, as it triggers selective elimination of mutant-derived chromosomes in hybrids with wild-type parents [karimi-ashtiyani_point_2015].' WHERE id=233


In [27]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])[
    "structure"
].values[0]

'The L92F mutation in barley βCENH3 (and its orthologous variants in other plants) disrupts centromeric histone loading, enabling the development of haploid inducer lines for accelerated breeding, as it triggers selective elimination of mutant-derived chromosomes in hybrids with wild-type parents [karimi-ashtiyani_point_2015].'

In [28]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])[
    "genes"
].values[0]

'While most diploid eukaryotes and flowering plants maintain only one CENH3 gene copy (even after whole-genome duplications), some diploid species like Arabidopsis lyrata, barley, rye, pea, and related legumes retain two functional CENH3 homologs [ishii_unequal_2020]. It was demonstrated that the duplication of HTR12 in A. lyrata may have facilitated adaptation to multiple centromeric satellite sequences, though the underlying mechanisms of this process require further investigation [kawabe_duplication_2006]. Pisum and closely related Lathyrus species also retain two CenH3 paralogs (CenH3-1 and CenH3-2) originating from a Fabeae ancestor [neumann_centromeres_2015, neumann_stretching_2012]. Despite divergence, they colocalize in centromeres, likely enabling meta-polycentric organization with multiple functional domains. CenH3-2 is highly conserved but shows adaptive evolution in its histone fold, while CenH3-1 is more variable (especially N-terminal) without signs of positive selection 

In [29]:
# Make sure data is committed to the database
conn.commit()

# <span style="color:black">Add summary description of cenH3_(Mammalia):</span>

```
cenH3_(Mammalia) is a centromere-specific histone variant in mammals, often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation. For a general description see cenH3_(Animals) class.
```

# <span style="color:black">Add to gene description of cenH3_(Mammalia):</span>

```
Eleven centromeric histone CENP-A genes were identified in cattle (Bos taurus), among which only CENP-A-L-1 appears to be functionally active, while the remaining members likely represent nonfunctional pseudogenes [li_identification_2008].
```

In [30]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Mammalia)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Mammalia),variant,Homo sapiens,9606,130,cenH3_(Animals),130,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [33]:
summary_desc = "cenH3_(Mammalia) is a centromere-specific histone variant in mammals, often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation. For a general description see cenH3_(Animals) class."
gene_desc = "Eleven centromeric histone CENP-A genes were identified in cattle (Bos taurus), among which only CENP-A-L-1 appears to be functionally active, while the remaining members likely represent nonfunctional pseudogenes [li_identification_2008]."
query = f"UPDATE histone_description SET summary='{summary_desc}', genes='{gene_desc}' WHERE id=130"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary='cenH3_(Mammalia) is a centromere-specific histone variant in mammals, often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation. For a general description see cenH3_(Animals) class.', genes='Eleven centromeric histone CENP-A genes were identified in cattle (Bos taurus), among which only CENP-A-L-1 appears to be functionally active, while the remaining members likely represent nonfunctional pseudogenes [li_identification_2008].' WHERE id=130


In [35]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Mammalia),variant,Homo sapiens,9606,130,cenH3_(Animals),130,cenH3_(Mammalia) is a centromere-specific hist...,null,Eleven centromeric histone CENP-A genes were i...,...,null,null,null,null,null,null,null,null,null,None


In [36]:
# Make sure data is committed to the database
conn.commit()

# <span style="color:black">Add summary description of cenH3_(Animals):</span>

```
cenH3_(Animals) is a centromere-specific histone variant in animals (Metazoa), often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation.
```

# <span style="color:black">Add to function description of cenH3_(Animals):</span>

```
CENP-A nucleosomes are essential for chromosome segregation. A functional kinetochore interacts with active centromeric chromatin reached with CENP-A to form the mitotic spindle. Notably, in certain holocentric organisms, CENP-A appears dispensable for meiotic chromosome segregation while remaining essential for mitosis. As demonstrated in Caenorhabditis elegans, CENP-A loading is specifically eliminated following meiosis I, and RNAi-mediated depletion of CENP-A during meiosis fails to disrupt proper chromosome segregation [monen_differential_2005]. This stands in striking contrast to mitotic divisions, where CENP-A is absolutely required for kinetochore assembly and faithful chromosome segregation. Moreover, C. elegans and C. remanei possess a second cenH3 histone gene that likely performs specialized functions distinct from canonical CENP-A roles [monen_separase_2015]. For more detailed information see the "gene" section.
```

# <span style="color:black">Add to gene description of cenH3_(Animals):</span>

```
The nematodes Caenorhabditis elegans and C. remanei possess two genes encoding the centromeric histone H3 variant (cenH3): HCP-3 (also known as CeCENP-A) and CPAR-1 [monen_separase_2015, monen_differential_2005]. While HCP-3 functions as the predominant centromeric histone - being highly expressed and essential for precise chromosome segregation during mitotic divisions - CPAR-1 displays distinct characteristics: it exhibits lower expression levels but shows specific enrichment on meiotic chromosomes. Importantly, CPAR-1 undergoes separase-mediated proteolytic cleavage at the meiosis I metaphase-to-anaphase transition, implying a specialized, though not yet fully characterized, role in meiotic processes [monen_separase_2015]. 
```

In [37]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Animals)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Animals),variant,Homo sapiens,9606,94,cenH3,94,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [40]:
summary_desc = "cenH3_(Animals) is a centromere-specific histone variant in animals (Metazoa), often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation."
gene_desc = 'CENP-A nucleosomes are essential for chromosome segregation. A functional kinetochore interacts with active centromeric chromatin reached with CENP-A to form the mitotic spindle. Notably, in certain holocentric organisms, CENP-A appears dispensable for meiotic chromosome segregation while remaining essential for mitosis. As demonstrated in Caenorhabditis elegans, CENP-A loading is specifically eliminated following meiosis I, and RNAi-mediated depletion of CENP-A during meiosis fails to disrupt proper chromosome segregation [monen_differential_2005]. This stands in striking contrast to mitotic divisions, where CENP-A is absolutely required for kinetochore assembly and faithful chromosome segregation. Moreover, C. elegans and C. remanei possess a second cenH3 histone gene that likely performs specialized functions distinct from canonical CENP-A roles [monen_separase_2015]. For more detailed information see the "gene" section.'
function_desc = "The nematodes Caenorhabditis elegans and C. remanei possess two genes encoding the centromeric histone H3 variant (cenH3): HCP-3 (also known as CeCENP-A) and CPAR-1 [monen_separase_2015, monen_differential_2005]. While HCP-3 functions as the predominant centromeric histone - being highly expressed and essential for precise chromosome segregation during mitotic divisions - CPAR-1 displays distinct characteristics: it exhibits lower expression levels but shows specific enrichment on meiotic chromosomes. Importantly, CPAR-1 undergoes separase-mediated proteolytic cleavage at the meiosis I metaphase-to-anaphase transition, implying a specialized, though not yet fully characterized, role in meiotic processes [monen_separase_2015]."
query = f"UPDATE histone_description SET summary='{summary_desc}', genes='{gene_desc}', function='{function_desc}' WHERE id=94"
print(query)
cursor.execute(query)

UPDATE histone_description SET summary='cenH3_(Animals) is a centromere-specific histone variant in animals (Metazoa), often called CENP-A (Centromere Protein A), and an important component of active centromere required for chromosome segregation.', genes='CENP-A nucleosomes are essential for chromosome segregation. A functional kinetochore interacts with active centromeric chromatin reached with CENP-A to form the mitotic spindle. Notably, in certain holocentric organisms, CENP-A appears dispensable for meiotic chromosome segregation while remaining essential for mitosis. As demonstrated in Caenorhabditis elegans, CENP-A loading is specifically eliminated following meiosis I, and RNAi-mediated depletion of CENP-A during meiosis fails to disrupt proper chromosome segregation [monen_differential_2005]. This stands in striking contrast to mitotic divisions, where CENP-A is absolutely required for kinetochore assembly and faithful chromosome segregation. Moreover, C. elegans and C. remane

In [41]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cenH3_(Animals)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cenH3_(Animals),variant,Homo sapiens,9606,94,cenH3,94,cenH3_(Animals) is a centromere-specific histo...,null,CENP-A nucleosomes are essential for chromosom...,...,null,The nematodes Caenorhabditis elegans and C. re...,null,null,null,null,null,null,null,None


In [42]:
# Make sure data is committed to the database
conn.commit()

# Add publications for cenH3_(Plants)

kawabe_duplication_2006, neumann_centromeres_2015, neumann_stretching_2012, sanei_loss_2011, karimi-ashtiyani_point_2015, ishii_differential_2015, evtushenko_conserved_2017

In [160]:
pids = [
    "kawabe_duplication_2006",
    "neumann_centromeres_2015",
    "neumann_stretching_2012",
    "sanei_loss_2011",
    "karimi-ashtiyani_point_2015",
    "ishii_differential_2015",
    "evtushenko_conserved_2017",
]

## There are some publications with id from pubmed

In [44]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_has_publication hp "
    "ON h.id = hp.histone_id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,histone_id,publication_id
0,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),14583738
1,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),19766562
2,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),21743476
3,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),22650316
4,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),23324462
5,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),25956076


In [50]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(df["publication_id"])]

,id,title,doi,author,year,histone_id,publication_id
24,14583738,None,None,None,None,cenH3_(Plants),14583738
39,19766562,None,None,None,None,cenH3_(Plants),19766562
47,21743476,None,None,None,None,cenH3_(Plants),21743476
54,22650316,None,None,None,None,cenH3_(Plants),22650316
55,22650316,None,None,None,None,cH4,22650316
56,22650316,None,None,None,None,generic_H1,22650316
57,22650316,None,None,None,None,H1.0,22650316
58,22650316,None,None,None,None,H1.10,22650316
59,22650316,None,None,None,None,H2A.L_(Eutheria),22650316
60,22650316,None,None,None,None,H2A.X,22650316


## Adding new feild pubmed_id to publication table in database

In [51]:
query = "ALTER TABLE publication ADD COLUMN pubmed_id varchar(50)"
cursor.execute(query)

In [52]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(df["publication_id"])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id
24,14583738,None,None,None,None,None,cenH3_(Plants),14583738
39,19766562,None,None,None,None,None,cenH3_(Plants),19766562
47,21743476,None,None,None,None,None,cenH3_(Plants),21743476
54,22650316,None,None,None,None,None,cenH3_(Plants),22650316
55,22650316,None,None,None,None,None,cH4,22650316
56,22650316,None,None,None,None,None,generic_H1,22650316
57,22650316,None,None,None,None,None,H1.0,22650316
58,22650316,None,None,None,None,None,H1.10,22650316
59,22650316,None,None,None,None,None,H2A.L_(Eutheria),22650316
60,22650316,None,None,None,None,None,H2A.X,22650316


In [53]:
# Make sure data is committed to the database
conn.commit()

## Correcting all previously added publications to cenH3_(Plants)

In [124]:
df["publication_id"]

0    14583738
1    19766562
2    21743476
3    22650316
4    23324462
5    25956076
Name: publication_id, dtype: object

### Set pubmed_id to these publications

In [125]:
for pubmed_id in df["publication_id"]:
    query = f"UPDATE publication SET pubmed_id='{pubmed_id}' WHERE id='{pubmed_id}'"
    print(query)
    cursor.execute(query)

UPDATE publication SET pubmed_id='14583738' WHERE id='14583738'
UPDATE publication SET pubmed_id='19766562' WHERE id='19766562'
UPDATE publication SET pubmed_id='21743476' WHERE id='21743476'
UPDATE publication SET pubmed_id='22650316' WHERE id='22650316'
UPDATE publication SET pubmed_id='23324462' WHERE id='23324462'
UPDATE publication SET pubmed_id='25956076' WHERE id='25956076'


In [126]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(df["publication_id"])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id
24,14583738,None,None,None,None,14583738,cenH3_(Plants),14583738
39,19766562,None,None,None,None,19766562,cenH3_(Plants),19766562
47,21743476,None,None,None,None,21743476,cenH3_(Plants),21743476
54,22650316,None,None,None,None,22650316,cenH3_(Plants),22650316
55,22650316,None,None,None,None,22650316,cH4,22650316
56,22650316,None,None,None,None,22650316,generic_H1,22650316
57,22650316,None,None,None,None,22650316,H1.0,22650316
58,22650316,None,None,None,None,22650316,H1.10,22650316
59,22650316,None,None,None,None,22650316,H2A.L_(Eutheria),22650316
60,22650316,None,None,None,None,22650316,H2A.X,22650316


In [127]:
query = (
    "SELECT * FROM publication p LEFT JOIN sequence_has_publication sh "
    "ON p.id = sh.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(df["publication_id"])]

,id,title,doi,author,year,pubmed_id,sequence_accession,publication_id
73,14583738,None,None,None,None,14583738,None,None
100,19766562,None,None,None,None,19766562,None,None
112,21743476,None,None,None,None,21743476,None,None
119,22650316,None,None,None,None,22650316,AAC28846.1,22650316
120,22650316,None,None,None,None,22650316,AAC37291.1,22650316
...,...,...,...,...,...,...,...,...
258,22650316,None,None,None,None,22650316,XP_846259.1,22650316
259,22650316,None,None,None,None,22650316,XP_875023.2,22650316
262,23324462,None,None,None,None,23324462,NP_001035891.1,23324462
263,23324462,None,None,None,None,23324462,NP_001800.1,23324462


### Create correct info for these publications

In [128]:
publications_info = {
    "14583738": {
        "id": "malik_phylogenomics_2003",
        "title": "Phylogenomics of the nucleosome",
        "doi": "10.1038/nsb996",
        "author": "",
        "year": "2003",
    },
    "19766562": {
        "id": "malik_major_2009",
        "title": "Major evolutionary transitions in centromere complexity",
        "doi": "10.1016/j.cell.2009.08.036",
        "author": "",
        "year": "2009",
    },
    "21743476": {
        "id": "tachiwana_crystal_2011",
        "title": "Crystal structure of the human centromeric nucleosome containing CENP-A",
        "doi": "10.1038/nature10258",
        "author": "",
        "year": "2011",
    },
    "22650316": {
        "id": "talbert_unified_2012",
        "title": "A unified phylogeny-based nomenclature for histone variants",
        "doi": "10.1186/1756-8935-5-7",
        "author": "",
        "year": "2012",
    },
    "23324462": {
        "id": "bui_cenp-nucleosome_2013",
        "title": "The CENP-A nucleosome: a battle between Dr Jekyll and Mr Hyde",
        "doi": "10.4161/nucl.23588",
        "author": "",
        "year": "2013",
    },
    "25956076": {
        "id": "steiner_diversity_2015",
        "title": "Diversity in the organization of centromeric chromatin",
        "doi": "10.1016/j.gde.2015.03.010",
        "author": "",
        "year": "2015",
    },
}

In [129]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin([publications_info[pmid]["id"] for pmid in df["publication_id"]])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id
228,malik_phylogenomics_2003,None,None,None,None,None,cH2A,malik_phylogenomics_2003
229,malik_phylogenomics_2003,None,None,None,None,None,cH2B,malik_phylogenomics_2003
230,malik_phylogenomics_2003,None,None,None,None,None,cH3,malik_phylogenomics_2003
231,malik_phylogenomics_2003,None,None,None,None,None,cH3_(Fungi),malik_phylogenomics_2003
232,malik_phylogenomics_2003,None,None,None,None,None,H2A,malik_phylogenomics_2003
233,malik_phylogenomics_2003,None,None,None,None,None,H2B,malik_phylogenomics_2003
234,malik_phylogenomics_2003,None,None,None,None,None,H3,malik_phylogenomics_2003
235,malik_phylogenomics_2003,None,None,None,None,None,H3.3,malik_phylogenomics_2003
236,malik_phylogenomics_2003,None,None,None,None,None,H3.3_(Fungi),malik_phylogenomics_2003
237,malik_phylogenomics_2003,None,None,None,None,None,H3.3_(Plants),malik_phylogenomics_2003


### Add histone relations for two of these publications that are already in DB - 14583738, 22650316

In [130]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(["14583738", "22650316"])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id
24,14583738,None,None,None,None,14583738,cenH3_(Plants),14583738
54,22650316,None,None,None,None,22650316,cenH3_(Plants),22650316
55,22650316,None,None,None,None,22650316,cH4,22650316
56,22650316,None,None,None,None,22650316,generic_H1,22650316
57,22650316,None,None,None,None,22650316,H1.0,22650316
58,22650316,None,None,None,None,22650316,H1.10,22650316
59,22650316,None,None,None,None,22650316,H2A.L_(Eutheria),22650316
60,22650316,None,None,None,None,22650316,H2A.X,22650316
61,22650316,None,None,None,None,22650316,H2A.Z,22650316
62,22650316,None,None,None,None,22650316,H3.4_(Mammalia),22650316


In [131]:
for i, row in dfp[dfp["id"].isin(["14583738", "22650316"])].iterrows():
    cursor.execute(
        add_histone_has_publication,
        (row["histone_id"], publications_info[row["publication_id"]]["id"]),
    )

In [132]:
query = (
    "SELECT * FROM publication p LEFT JOIN sequence_has_publication sh "
    "ON p.id = sh.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(["14583738", "22650316"])]

,id,title,doi,author,year,pubmed_id,sequence_accession,publication_id
73,14583738,None,None,None,None,14583738,None,None
119,22650316,None,None,None,None,22650316,AAC28846.1,22650316
120,22650316,None,None,None,None,22650316,AAC37291.1,22650316
121,22650316,None,None,None,None,22650316,AAC37292.1,22650316
122,22650316,None,None,None,None,22650316,CAA33554.1,22650316
...,...,...,...,...,...,...,...,...
255,22650316,None,None,None,None,22650316,XP_627120.1,22650316
256,22650316,None,None,None,None,22650316,XP_641587.1,22650316
257,22650316,None,None,None,None,22650316,XP_845905.1,22650316
258,22650316,None,None,None,None,22650316,XP_846259.1,22650316


In [133]:
for i, row in dfp[dfp["id"].isin(["14583738", "22650316"])].iterrows():
    if row["publication_id"] is None:
        continue
    cursor.execute(
        add_sequence_has_publication,
        (row["sequence_accession"], publications_info[row["publication_id"]]["id"]),
    )

In [134]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin([publications_info[pmid]["id"] for pmid in df["publication_id"]])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id
228,malik_phylogenomics_2003,None,None,None,None,None,cenH3_(Plants),malik_phylogenomics_2003
229,malik_phylogenomics_2003,None,None,None,None,None,cH2A,malik_phylogenomics_2003
230,malik_phylogenomics_2003,None,None,None,None,None,cH2B,malik_phylogenomics_2003
231,malik_phylogenomics_2003,None,None,None,None,None,cH3,malik_phylogenomics_2003
232,malik_phylogenomics_2003,None,None,None,None,None,cH3_(Fungi),malik_phylogenomics_2003
233,malik_phylogenomics_2003,None,None,None,None,None,H2A,malik_phylogenomics_2003
234,malik_phylogenomics_2003,None,None,None,None,None,H2B,malik_phylogenomics_2003
235,malik_phylogenomics_2003,None,None,None,None,None,H3,malik_phylogenomics_2003
236,malik_phylogenomics_2003,None,None,None,None,None,H3.3,malik_phylogenomics_2003
237,malik_phylogenomics_2003,None,None,None,None,None,H3.3_(Fungi),malik_phylogenomics_2003


In [135]:
query = (
    "SELECT * FROM publication p LEFT JOIN sequence_has_publication sh "
    "ON p.id = sh.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin([publications_info[pmid]["id"] for pmid in df["publication_id"]])]

,id,title,doi,author,year,pubmed_id,sequence_accession,publication_id
1040,malik_phylogenomics_2003,None,None,None,None,None,None,None
5890,talbert_unified_2012,None,None,None,None,None,AAC28846.1,talbert_unified_2012
5891,talbert_unified_2012,None,None,None,None,None,AAC37291.1,talbert_unified_2012
5892,talbert_unified_2012,None,None,None,None,None,AAC37292.1,talbert_unified_2012
5893,talbert_unified_2012,None,None,None,None,None,CAA33554.1,talbert_unified_2012
...,...,...,...,...,...,...,...,...
6026,talbert_unified_2012,None,None,None,None,None,XP_627120.1,talbert_unified_2012
6027,talbert_unified_2012,None,None,None,None,None,XP_641587.1,talbert_unified_2012
6028,talbert_unified_2012,None,None,None,None,None,XP_845905.1,talbert_unified_2012
6029,talbert_unified_2012,None,None,None,None,None,XP_846259.1,talbert_unified_2012


### Delete two relations for these publications

In [136]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(["14583738", "22650316"])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id
24,14583738,None,None,None,None,14583738,cenH3_(Plants),14583738
54,22650316,None,None,None,None,22650316,cenH3_(Plants),22650316
55,22650316,None,None,None,None,22650316,cH4,22650316
56,22650316,None,None,None,None,22650316,generic_H1,22650316
57,22650316,None,None,None,None,22650316,H1.0,22650316
58,22650316,None,None,None,None,22650316,H1.10,22650316
59,22650316,None,None,None,None,22650316,H2A.L_(Eutheria),22650316
60,22650316,None,None,None,None,22650316,H2A.X,22650316
61,22650316,None,None,None,None,22650316,H2A.Z,22650316
62,22650316,None,None,None,None,22650316,H3.4_(Mammalia),22650316


In [137]:
for del_id in ["14583738", "22650316"]:
    query = f"DELETE FROM histone_has_publication WHERE publication_id='{del_id}' "
    cursor.execute(query)
    query = f"DELETE FROM sequence_has_publication WHERE publication_id='{del_id}' "
    cursor.execute(query)
    query = f"DELETE FROM publication WHERE id='{del_id}' "
    cursor.execute(query)

In [140]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(["14583738", "22650316"])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id


In [139]:
query = (
    "SELECT * FROM publication p LEFT JOIN histone_has_publication ph "
    "ON p.id = ph.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin([publications_info[pmid]["id"] for pmid in df["publication_id"]])]

,id,title,doi,author,year,pubmed_id,histone_id,publication_id
212,malik_phylogenomics_2003,None,None,None,None,None,cenH3_(Plants),malik_phylogenomics_2003
213,malik_phylogenomics_2003,None,None,None,None,None,cH2A,malik_phylogenomics_2003
214,malik_phylogenomics_2003,None,None,None,None,None,cH2B,malik_phylogenomics_2003
215,malik_phylogenomics_2003,None,None,None,None,None,cH3,malik_phylogenomics_2003
216,malik_phylogenomics_2003,None,None,None,None,None,cH3_(Fungi),malik_phylogenomics_2003
217,malik_phylogenomics_2003,None,None,None,None,None,H2A,malik_phylogenomics_2003
218,malik_phylogenomics_2003,None,None,None,None,None,H2B,malik_phylogenomics_2003
219,malik_phylogenomics_2003,None,None,None,None,None,H3,malik_phylogenomics_2003
220,malik_phylogenomics_2003,None,None,None,None,None,H3.3,malik_phylogenomics_2003
221,malik_phylogenomics_2003,None,None,None,None,None,H3.3_(Fungi),malik_phylogenomics_2003


In [141]:
# Make sure data is committed to the database
conn.commit()

### Change ids for other publications

In [143]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_has_publication hp "
    "ON h.id = hp.histone_id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,histone_id,publication_id
0,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),19766562
1,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),21743476
2,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),23324462
3,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),25956076
4,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),malik_phylogenomics_2003
5,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),talbert_unified_2012


In [142]:
df["publication_id"]

0    14583738
1    19766562
2    21743476
3    22650316
4    23324462
5    25956076
Name: publication_id, dtype: object

In [147]:
query = (
    "SELECT * FROM publication p LEFT JOIN sequence_has_publication sh "
    "ON p.id = sh.publication_id "
)
cursor.execute(query)
dfp = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
dfp[dfp["id"].isin(df["publication_id"])]

,id,title,doi,author,year,pubmed_id,sequence_accession,publication_id
99,19766562,None,None,None,None,19766562,None,None
111,21743476,None,None,None,None,21743476,None,None
120,23324462,None,None,None,None,23324462,NP_001035891.1,23324462
121,23324462,None,None,None,None,23324462,NP_001800.1,23324462
171,25956076,None,None,None,None,25956076,None,None


In [150]:
query = "SELECT * FROM sequence_has_publication " "WHERE publication_id = '23324462' "
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,sequence_accession,publication_id
0,NP_001035891.1,23324462
1,NP_001800.1,23324462


In [151]:
two_accessions = ["NP_001035891.1", "NP_001800.1"]
query = f"DELETE FROM sequence_has_publication WHERE publication_id='23324462' "
cursor.execute(query)

In [152]:
for pubmed_id in df["publication_id"]:
    if pubmed_id in ["14583738", "22650316"]:
        continue
    query = f"UPDATE publication SET id='{publications_info[pubmed_id]['id']}', title='{publications_info[pubmed_id]['title']}', doi='{publications_info[pubmed_id]['doi']}', year='{publications_info[pubmed_id]['year']}' WHERE id='{pubmed_id}'"
    print(query)
    cursor.execute(query)

UPDATE publication SET id='malik_major_2009', title='Major evolutionary transitions in centromere complexity', doi='10.1016/j.cell.2009.08.036', year='2009' WHERE id='19766562'
UPDATE publication SET id='tachiwana_crystal_2011', title='Crystal structure of the human centromeric nucleosome containing CENP-A', doi='10.1038/nature10258', year='2011' WHERE id='21743476'
UPDATE publication SET id='bui_cenp-nucleosome_2013', title='The CENP-A nucleosome: a battle between Dr Jekyll and Mr Hyde', doi='10.4161/nucl.23588', year='2013' WHERE id='23324462'
UPDATE publication SET id='steiner_diversity_2015', title='Diversity in the organization of centromeric chromatin', doi='10.1016/j.gde.2015.03.010', year='2015' WHERE id='25956076'


In [156]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_has_publication hp "
    "ON h.id = hp.histone_id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,histone_id,publication_id
0,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),bui_cenp-nucleosome_2013
1,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),malik_major_2009
2,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),malik_phylogenomics_2003
3,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),steiner_diversity_2015
4,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),tachiwana_crystal_2011
5,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),talbert_unified_2012


In [157]:
for a in two_accessions:
    cursor.execute(
        add_sequence_has_publication,
        (a, publications_info["23324462"]["id"]),
    )

In [158]:
query = (
    "SELECT * FROM sequence_has_publication "
    f"WHERE publication_id = '{publications_info['23324462']['id']}' "
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,sequence_accession,publication_id
0,NP_001035891.1,bui_cenp-nucleosome_2013
1,NP_001800.1,bui_cenp-nucleosome_2013


In [159]:
# Make sure data is committed to the database
conn.commit()

## Add publications for cenH3_(Plants)

kawabe_duplication_2006, neumann_centromeres_2015, neumann_stretching_2012, sanei_loss_2011, karimi-ashtiyani_point_2015, ishii_differential_2015, evtushenko_conserved_2017

In [161]:
pids

['kawabe_duplication_2006',
 'neumann_centromeres_2015',
 'neumann_stretching_2012',
 'sanei_loss_2011',
 'karimi-ashtiyani_point_2015',
 'ishii_differential_2015',
 'evtushenko_conserved_2017']

In [162]:
query = f"SELECT * FROM publication"
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].isin(pids)]

,id,title,doi,author,year,pubmed_id


In [165]:
data_publication = [
    {
        "id": pid,
        "title": None,
        "doi": None,
        "author": None,
        "year": None,
    }
    for pid in pids
]

In [166]:
for dp in data_publication:
    cursor.execute(add_publication, dp)

In [167]:
for pid in pids:
    cursor.execute(add_histone_has_publication, ("cenH3_(Plants)", pid))

In [168]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_has_publication hp "
    "ON h.id = hp.histone_id "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,histone_id,publication_id
0,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),bui_cenp-nucleosome_2013
1,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),evtushenko_conserved_2017
2,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),ishii_differential_2015
3,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),karimi-ashtiyani_point_2015
4,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),kawabe_duplication_2006
5,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),malik_major_2009
6,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),malik_phylogenomics_2003
7,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),neumann_centromeres_2015
8,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),neumann_stretching_2012
9,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,cenH3_(Plants),sanei_loss_2011


In [169]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [170]:
cursor.close()
conn.close()
tunnel.stop()